In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
#mount google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

# Use the GPU instead of CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [ ]:
# Pull in the data from CSV files
train_data = pd.read_csv('/content/drive/My Drive/train_data.csv')
train_target = pd.read_csv('/content/drive/My Drive/train_target.csv')
test_data = pd.read_csv('/content/drive/My Drive/test_data.csv')

# print shape to verify successful import
print("Train Data : " + str(train_data.shape))
print("Train Data Target : " + str(train_target.shape))
print("Test Data : " + str(test_data.shape))

Train Data : (16174, 2304)
Train Data Target : (16174, 1)
Test Data : (3964, 2304)


In [ ]:
# Data Preprocessing
# Convert the data to NumPy arrays and reshape them to image dimensions (48x48)
train_images = np.array(train_data.values, dtype='uint8').reshape(-1, 1, 48, 48)
train_labels = np.array(train_target.values, dtype='int64').squeeze()
test_images = np.array(test_data.values, dtype='uint8').reshape(-1, 1, 48, 48)

# Normalize the pixel values to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert data to PyTorch tensors
train_images = torch.tensor(train_images, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.int64)
test_images = torch.tensor(test_images, dtype=torch.float32)

In [ ]:
class FacialExpressionDataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        if self.transform:
            # If the image is not a tensor, apply the transform
            if not torch.is_tensor(image):
                image = self.transform(image)
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image  # Return only the image for the test set


In [ ]:
# Data augmentation and transformation for training dataset
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

# Apply transformation to training dataset
train_dataset = FacialExpressionDataset(train_images, train_labels, transform=train_transform)
# Test dataset does not require any transformation, so no need to specify a transform
test_dataset = FacialExpressionDataset(test_images)

In [ ]:
# Define hyperparameters
batch_size = 32
num_epochs = 20
learning_rate = 0.001

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define the CNN model
class FacialExpressionModel(nn.Module):
    def __init__(self):
        super(FacialExpressionModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 12 * 12, 128),
            nn.ReLU(),
            nn.Linear(128, 3)  # 3 output classes (Angry, Happy, Neutral)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the output from convolutional layers
        x = self.fc_layers(x)
        return x

In [ ]:
# Initialize the model and move it to the device (GPU or CPU)
model = FacialExpressionModel().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Print the average loss for each epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/20, Loss: 0.9946656854963114
Epoch 2/20, Loss: 0.8604100542106176
Epoch 3/20, Loss: 0.7955482696945017
Epoch 4/20, Loss: 0.7433649482345392
Epoch 5/20, Loss: 0.7091523840022181
Epoch 6/20, Loss: 0.6852335075262507
Epoch 7/20, Loss: 0.65933586415566
Epoch 8/20, Loss: 0.6356626164182844
Epoch 9/20, Loss: 0.6128647646649553
Epoch 10/20, Loss: 0.5877351085894664
Epoch 11/20, Loss: 0.5635497533521162
Epoch 12/20, Loss: 0.5432314073203109
Epoch 13/20, Loss: 0.5171991218279002
Epoch 14/20, Loss: 0.49862710554373596
Epoch 15/20, Loss: 0.46994125430763
Epoch 16/20, Loss: 0.44811054940515826
Epoch 17/20, Loss: 0.4229429862774879
Epoch 18/20, Loss: 0.39738762755638996
Epoch 19/20, Loss: 0.37931623831096845
Epoch 20/20, Loss: 0.3514785026844311


In [ ]:
# Evaluation on the test set
model.eval()  # Set the model to evaluation mode
predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

# Save predictions to a CSV file in the required format
submission_df = pd.DataFrame({'Id': range(len(predictions)), 'Category': predictions})
submission_df.to_csv('/content/drive/My Drive/facial_expression_predictions4.csv', index=False)